<a href="https://colab.research.google.com/github/Cerabbite/Easy-conalng/blob/main/Word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For each word in the top 10.000 most used English, Spanish, Japanese and Chinese words. There will be 100-1.000 sample sentences with those words (written in using the english alphabet). Using these sentences this script is going to work out how related each word is to another word.

In [2]:
# Import libraries
#!pip install keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json
import numpy as np

In [13]:
# Dataset to python
with open('Sarcasm_Headlines_Dataset_v2.json', 'r') as f:
  info = f.readlines()

datastore = []
for i in info:
  datastore.append(json.loads(i))


sentences = []
labels = []
urls = []
for item in datastore:
  sentences.append(item['headline'])
  labels.append(item['is_sarcastic'])
  urls.append(item['article_link'])

In [14]:
# Variables
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [15]:
# Seperate training from testing
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [16]:
# Tokenization
"""sentences = [
             'I love my dog',
             'I love my cat',
             'You love my dog!',
             'Do you think my dog is amazing?'
]"""

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

#print(word_index)
print(training_padded[0])
print(training_padded.shape)

[   1  325 3169 5817 2489    3  655  993    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
(20000, 100)


In [17]:
# Test
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(testing_padded[0])
print(testing_padded.shape)

[  83 4338    1    6 2186  137  625  181   31   28 3008   56    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
(8619, 100)


In [18]:
# Neural network
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                             tf.keras.layers.GlobalAveragePooling1D(),
                             tf.keras.layers.Dense(24, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

num_epochs = 50

history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/50
625/625 - 4s - loss: 0.6525 - accuracy: 0.6210 - val_loss: 0.5117 - val_accuracy: 0.8015 - 4s/epoch - 6ms/step
Epoch 2/50
625/625 - 2s - loss: 0.3981 - accuracy: 0.8389 - val_loss: 0.3650 - val_accuracy: 0.8426 - 2s/epoch - 4ms/step
Epoch 3/50
625/625 - 2s - loss: 0.3022 - accuracy: 0.8778 - val_loss: 0.3478 - val_accuracy: 0.8459 - 2s/epoch - 4ms/step
Epoch 4/50
625/625 - 2s - loss: 0.2523 - accuracy: 0.9014 - val_loss: 0.3310 - val_accuracy: 0.8566 - 2s/epoch - 3ms/step
Epoch 5/50
625/625 - 2s - loss: 0.2181 - accuracy: 0.9179 - val_loss: 0.3337 - val_accuracy: 0.8550 - 2s/epoch - 3ms/step
Epoch 6/50
625/625 - 2s - loss: 0.1911 - accuracy: 0.9280 - val_loss: 0.3502 - val_accuracy: 0.8527 - 2s/epoch - 4ms/step
Epoch 7/50
625/625 - 2s - loss: 0.1698 - accuracy: 0.9376 - val_loss: 0.3644 - val_accuracy: 0.8494 - 2s/epoch - 4ms/step
Epoch 8/50
625/625 - 2s - loss: 0.1519 - accuracy: 0.9449 - val_loss: 0.3748 - val_accuracy: 0.8496 - 2s/epoch - 4ms/step
Epoch 9/50
625/625 - 3s 

In [19]:
# Use trained model
sentence = [
            "Granny starting to fear spiders in the garden might be real",
            "the weather today is bright and sunny",
            "That's just what I needed today!",
            "Well, what a surprise.",
            "Really, Sherlock? No! You are clever.",
            "Today I had a job interview.",
            "He went to grab some snacks."
]

sequences = tokenizer.texts_to_sequences(sentence)

padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(model.predict(padded))

[[9.99617934e-01]
 [5.64974880e-06]
 [9.99994516e-01]
 [1.08725915e-04]
 [4.13626432e-04]
 [6.87621196e-08]
 [6.43782854e-01]]
